# Seismic Data Processing

This tutorial aims to introduce some basic seismic data processing steps.

**Authors**

- Shijie Hao ([HouseJaay](https://github.com/HouseJaay))
- Jiayuan Yao ([core-man](https://github.com/core-man))

## Preparation

We first use [ObsPy mass_downloader](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html) to download seismic data recorded at seismic stations in a box region for the [Ridgecreat earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/ci38457511/executive) sequence.

To save time, we only use three largest ones, including the main shock, a foreshock and a aftershock. Actually, this catalog is just the exercise last week.

## Introduction

After we download raw seismic time series data from data centers, we usually perform some basic data processing before further data analysis.

There are several basic data processing steps which are commonly performed in the daily research.

- [SAC Chinese Documention](https://seisman.github.io/SAC_Docs_zh/data-process/)
- [ObsPy Tutorial](https://docs.obspy.org/master/tutorial/index.html)
- [Live Jupyter Notebooks for Seismology](https://krischer.github.io/seismo_live_build/tree/index.html)
- [Remote Online Sessions for Emerging Seismologists: ObsPy](https://www.iris.edu/hq/inclass/course/roses)


In this tutorial, we only introduce some of them indicated below, which must be performed in our seismic tomographic studies, while other data processing may be needed in your own studies.

- Time series data format conversion
- Data merging
- Data file renaming
- Adding event and station metadata
- Filtering

While we use ObsPy to present data processing here, you are encouraged to refer to [SAC Chinese Documentation](https://seisman.github.io/SAC_Docs_zh/call-in-script/) to learn how to call SAC in Python/bash/Perl to perform the same processing.

In [39]:
from obspy import UTCDateTime
import obspy
import obspy.io.sac.sactrace
from obspy.taup import TauPyModel
from distaz import DistAz
import os
import glob

## Define some tool functions

You can define your own tools functions according to your data.

### Read event metadata

The used earthquake catalog is a csv file. We may use [csv module](https://docs.python.org/3/library/csv.html) to simplify the code.

In [40]:
def read_evt(evtf):
    """
    read event information
    """
    with open(evtf, 'r') as f:
        lines = f.readlines()[1:]
    events = []
    dnames = []
    for line in lines:
        line = line.strip()
        temp = line.split(',')
        t = UTCDateTime(temp[0])
        lat, lon = float(temp[1]), float(temp[2])
        depth = float(temp[3])
        dpu = temp[4]
        mag = float(temp[5])
        magt = temp[6]
        events.append([t, lat, lon, depth, dpu, mag, magt])
        dnames.append("%04d%02d%02d%02d%02d%02d%s" % (t.year, t.month, t.day, t.hour, t.minute, t.second, str(t.microsecond)[:3]))
    return dnames, events

### Read station metadata

The used station metadata is a custom file. We may revise the code to use other format.

**Note** that the dip of an instrument in SOD and ObsPy are a little different from that defined in SAC.

In [41]:
def read_sta(staf):
    """
    read station information
    """
    with open(staf, 'r') as f:
        lines = f.readlines()
    stations, name = [], []
    for line in lines:
        line=line.strip()
        #if (not line) or line[0] == '#':
        #    continue

        #temp = line.split(' ')
        temp = line.split('|')
        name.append(temp[0])
        stla, stlo, stel, azimuth, dip = float(temp[1]), float(temp[2]), float(temp[3].split()[0]), float(temp[5]), float(temp[6])
        stations.append([stla, stlo, stel, azimuth, dip])
    return name, stations

## Data pre-processing

Some data pre-processing have to be performed, since the downloaded time series data are miniseed.

- Time series data format conversion:
    - `Waveform Import/Export Plug-ins` in [ObsPy](https://docs.obspy.org/master/packages/index.html): e.g., [Python interface to SAC](https://docs.obspy.org/packages/autogen/obspy.io.sac.sactrace.html) ([SACTrace class](https://docs.obspy.org/packages/autogen/obspy.io.sac.sactrace.SACTrace.html))
    - [rdseed](https://seisman.github.io/SAC_Docs_zh/data-process/data-format/)
    - [mseed2sac](https://ds.iris.edu/ds/nodes/dmc/software/downloads/mseed2sac/)
    - [sac2tol](https://github.com/core-man/SACtools#sac2col)
    
- `merge`: [ObsPy](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.merge.html) | [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/merge-traces/)

- Data file renmaing: [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/rename/)

- Adding event and station metadata:
    - [obspy.io.sac](https://docs.obspy.org/master/packages/obspy.io.sac.html) | [Python interface to SAC](https://docs.obspy.org/packages/autogen/obspy.io.sac.sactrace.html) ([SACTrace class](https://docs.obspy.org/packages/autogen/obspy.io.sac.sactrace.SACTrace.html))
    - SAC: [event](https://seisman.github.io/SAC_Docs_zh/data-process/event-info/) | [station](https://seisman.github.io/SAC_Docs_zh/data-process/station-info/)

In [42]:
def mseed2sac(data_root, data_out, dnames, events, fnames, stations):
    """
    convert miniseed to SAC
    """
    
    for evt in zip(dnames, events):
        # output dir
        outd = os.path.join(data_out, evt[0])
        if not os.path.exists(outd):
            os.makedirs(outd)

        for sta in zip(fnames, stations):
            # read miniseed
            fpath = os.path.join(data_root, evt[0], sta[0] + '.mseed')
            try:
                st = obspy.read(fpath)
            except FileNotFoundError:
                #print("Missing File: %s" % fpath)
                continue

            # merge multiple traces
            st.merge(method=1, fill_value=0, interpolation_samples=-1)
            
            # convert to SACTrace from ObsPy Trace
            sactr = obspy.io.sac.SACTrace.from_obspy_trace(st[0])
            
            # set event origin time as SAC reference time
            sactr.reftime = evt[1][0]
            sactr.o = 0
            sactr.iztype = 'io'

            # set station location
            sactr.stla = sta[1][0]
            sactr.stlo = sta[1][1]
            sactr.stel = sta[1][2]
            
            # set event location and magnitude
            sactr.evla = evt[1][1]
            sactr.evlo = evt[1][2]
            sactr.evdp = evt[1][3]
            sactr.mag = evt[1][5]
            
            # set cmpaz and cmpinc
            sactr.cmpaz = sta[1][3]
            sactr.cmpinc = sta[1][4] + 90.0
            
            # other SAC headers
            sactr.lcalda = 1  # DIST AZ BAZ and GCARC headers will be calculated from station and event coordinates.
                              # Note that thing may not be same if we use ObsPy Trace.
            
            # write to sac files
            sactr.write(os.path.join(outd, sta[0]+'.SAC'))

## Basic data processing

Now we begin to perform some basic data processing. In this tutorial, we only do **`rmean` | `rtrend` | `taper`:** and `filtering`, while you can conduct other processing based on the references.


**`rmean` | `rtrend` | `taper`:**

- ObsPy: [detrend](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.detrend.html) | [taper](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.taper.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/rmean-rtrend-taper/)

**Remove instrument response:**

- ObsPy: [remove_response](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.remove_response.html) | [remove sensitivity](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.remove_sensitivity.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/instrument-response/)

**Trim data:**

- [ObsPy](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.trim.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/cut-data/)

**Coordinate systerm rotation:**

- [IRISWS roation](https://service.iris.edu/irisws/rotation/docs/1/help/)
- ObsPy: [Stream.rotate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.rotate.html) | [obspy.singal.rotate](https://docs.obspy.org/packages/autogen/obspy.signal.rotate.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/rotate/)

**Resampling:**

- ObsPy: [resample](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.resample.html) | [decimate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.decimate.html) | [interpolate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.interpolate.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/resampling/)

**Filtering:**

- [ObsPy](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.filter.html)
- [SAC](https://seisman.github.io/SAC_Docs_zh/data-process/filter/)


**More methods can found be in ObsPy and SAC.**

- ObsPy: [Stream](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html) | [Trace](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.html) | [obspy.signal](https://docs.obspy.org/packages/obspy.signal.html)
- [Chinse SAC documentation](https://seisman.github.io/SAC_Docs_zh/data-process/).

### Rotate seismic time series data to different coordinate systems

Sometimes, we have to use data in `RTZ` coordinate sysmtem instead of the original one, e.g., `NEZ` or `12Z`.

In [43]:
def do_rotate(data_dir):
    """
    rotate data to RTZ coordinate system
    """
    pass

### Filter seismic time series data

Bandpass seismic time series data

In [44]:
def do_filter(data_dir, out_dir, freq_low, freq_high):
    """
    filter data
    """
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    for ev in os.listdir(data_dir):
        inpath = os.path.join(data_dir, ev)
        outpath = os.path.join(out_dir, ev)
        if not os.path.exists(outpath):
            os.mkdir(outpath)

        for sac in os.listdir(inpath):
            insac = os.path.join(inpath, sac)
            outsac = os.path.join(outpath, sac)
            st = obspy.read(insac)

            # rmean, rtrend & taper
            st.detrend("demean")
            st.detrend("linear")
            st.taper(max_percentage=0.05, type='hann')
            
            # bandpass
            st.filter('bandpass', freqmin=freq_low, freqmax=freq_high, corners=2)
            
            st[0].write(outsac, format='SAC')

## Data analysis

In this tutorial, some data analyses are performed here for the convenience. However, you should decide when and where you should perform those data analyses.

### Write theoretical arrival times

Calculate theoretical arrival times and write to SAC headers.

- [ObsPy](https://docs.obspy.org/packages/obspy.taup.html)
- [SAC Chinese Documentation](https://seisman.github.io/SAC_Docs_zh/data-process/traveltime/)

In [45]:
def theo_arrival(data_dir, model_name, phase):
    """
    Write theoretical arrival times
    """
    model = TauPyModel(model=model_name)

    for ev in os.listdir(data_dir):
        inpath = os.path.join(data_dir, ev)
        #print(inpath)

        for sacfile in glob.glob('{}/*.SAC'.format(inpath)):
            #print(sacfile)

            st = obspy.read(sacfile)
            sachd = st[0].stats.sac
            
            # set P-wave first arrival
            da = DistAz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
            gcarc = da.getDelta()
            arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                                  distance_in_degree=gcarc, phase_list=[phase])
            
            sachd["t1"] = arrivals[0].time + sachd["o"]
            sachd["kt1"] = arrivals[0].name
            
            st[0].write(sacfile, format="SAC")

## Begin to do data process

Call the above functions

In [46]:
# dirs and files
data_root = 'miniseed'                     # miniseed data dir
data_sac = 'sac/raw'                       # raw sac data dir
data_filt = 'sac/filt_1_4Hz'               # filtered sac data dir
evt_lst = 'events.csv'                     # event metadata
sta_lst = '../data-fetch/SOD/station.txt'  # station metadata

if not os.path.exists(data_sac):
    os.makedirs(data_sac)
    
# read event and station metadata
dnames, events = read_evt(evt_lst)
fnames, stations = read_sta(sta_lst)

# convert miniseed to sac
mseed2sac(data_root, data_sac, dnames, events, fnames, stations)

# write theoretical arrival times
theo_arrival(data_sac, "ak135", "ttp")

# filter data
do_filter(data_sac, data_filt, 1.0, 4.0)